# Geocoding Architecture Assessment

This notebook is designed to assess the performance and accuracy of the following geocoding process.

`Google Geocoding API with Raw Input`

Execution time is the sum of the run time to complete the standardization and geocoding of 100 sample records.

Performance is measured by several different metrics, including the Mean, Standard Deviation, RMSE, and MAE of error in the X and Y directions (Manhattan Distance), as well as the percentile error for 75, 90, and 95% of the Euclidean Distance. The percentile error analysis produces the threshold value which signifies the distance within which the predicted coordinates are considered accurate and the percentage value which indicates the portion of points that fall within the specified error threshold (75, 90, or 95).

Below are the results of the tests.

In [1]:
# Imports
import math
import os

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import requests

### Loading Data

In [2]:
# Ground Truth Data
truth_df = pd.read_csv(r"../../data/other/gc_research_truth.csv")

truth_df.head()

,index,index2,Type,ICR,Date,District,Location,RoadCondition,NumberVehicles,GroudTruth_X,GroudTruth_Y
0,558,558,Injury,22301608,08/10/2022 10:53,2300 Marshall,"WB Interstate 90 near Magnolia, MAGNOLIA TWP, ...",Dry,1 Vehicle involved.,-96.098900,43.637800
1,1288,1288,Fatal,21902377,12/03/2021 17:47,2900 Detroit Lakes,"210 Highway at County Road 35, UNDERWOOD, Otte...",Dry,2 Vehicle,-95.874370,46.281201
2,467,467,Injury,20320777,06/16/2020 14:54,3200 Thief River Falls,"Westbound Hwy 2 east of Hwy 92 outside Bagley,...",Dry,2 Vehicle,-95.398111,47.521845
3,544,544,Injury,22406181,08/13/2022 23:49,2400 East Metro,"NB 61 HWY to Innovation, COTTAGE GROVE, Washin...",Dry,1 Vehicle involved.,-92.910974,44.805176
4,478,478,Injury,23200293,02/01/2023 18:23,2200 Mankato,"Highway 169 milepost 23, WINNEBAGO CITY TWP, F...",Dry,1 Vehicle involved.,-94.168037,43.810902


In [3]:
# Raw Data
raw_df = pd.read_csv(r"../../data/other/gc_research_truth.csv")

raw_df = raw_df.drop(['GroudTruth_X', 'GroudTruth_Y'], axis = 1)

raw_df.head()

,index,index2,Type,ICR,Date,District,Location,RoadCondition,NumberVehicles
0,558,558,Injury,22301608,08/10/2022 10:53,2300 Marshall,"WB Interstate 90 near Magnolia, MAGNOLIA TWP, ...",Dry,1 Vehicle involved.
1,1288,1288,Fatal,21902377,12/03/2021 17:47,2900 Detroit Lakes,"210 Highway at County Road 35, UNDERWOOD, Otte...",Dry,2 Vehicle
2,467,467,Injury,20320777,06/16/2020 14:54,3200 Thief River Falls,"Westbound Hwy 2 east of Hwy 92 outside Bagley,...",Dry,2 Vehicle
3,544,544,Injury,22406181,08/13/2022 23:49,2400 East Metro,"NB 61 HWY to Innovation, COTTAGE GROVE, Washin...",Dry,1 Vehicle involved.
4,478,478,Injury,23200293,02/01/2023 18:23,2200 Mankato,"Highway 169 milepost 23, WINNEBAGO CITY TWP, F...",Dry,1 Vehicle involved.


### Pipeline Code

In [4]:
def geocode(df, description_series):
    df["xy"] = df[description_series].apply(
        _single_google_geocode
    )
    
    df["x"] = df["xy"].apply(
        lambda row: row.split(",")[0].strip()
    )
    df["y"] = df["xy"].apply(
        lambda row: row.split(",")[1].strip()
    )
    
    df["x"] = pd.to_numeric(df["x"], errors="coerce")
    df["y"] = pd.to_numeric(df["y"], errors="coerce")
    
    
    df = df[df["x"].notna()]
    df = df[df["y"].notna()]
    
    return df
    
def _single_google_geocode(input_text):
    google_key=os.getenv('GOOGLE_MAPS_KEY')
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    r = requests.get(
        geocode_url,
        params={
            "address": input_text,
            "key": google_key,
            "bounds": "43.00,-97.50 49.5,-89.00",
        },
    )

    j = r.json()

    try:
        x = j["results"][0]["geometry"]["location"]["lng"]
        y = j["results"][0]["geometry"]["location"]["lat"]
    
    except:
        x = None
        y = None

    return f"{x}, {y}"

In [5]:
# Accuracy Assessment Code
def accuracy_assessment(df):
    x_truth=list(df["GroudTruth_X"])
    y_truth=list(df["GroudTruth_Y"])
    x_pred=list(df["x"])
    y_pred=list(df["y"])
    
    # Calculate Deltas
    delta_x = []
    delta_y = []

    for i in range(len(x_truth)):
        delta_x.append(x_truth[i] - x_pred[i])
        delta_y.append(y_truth[i] - y_pred[i])

    # All Deltas
    deltas = delta_x + delta_y

    # MAE
    abs_deltas = [abs(i) for i in deltas]
    mae = sum(abs_deltas) / len(abs_deltas)

    # RMSE
    sq_deltas = [i**2 for i in deltas]
    rmse = math.sqrt((sum(sq_deltas) / len(sq_deltas)))

    # Mean
    mean = sum(deltas) / len(deltas)

    # Standard Deviation
    stdev_diffs = []
    for i in deltas:
        stdev_diffs.append(i - mean)

    stdev_diffs_sq = [i**2 for i in stdev_diffs]

    stdev = math.sqrt((sum(stdev_diffs_sq) / len(stdev_diffs_sq)))
    
    # Percentile Errors
    pe_75_et, pe_75_pwt = _percentile_error(75, x_truth, y_truth, x_pred, y_pred)
    pe_90_et, pe_90_pwt = _percentile_error(90, x_truth, y_truth, x_pred, y_pred)
    pe_95_et, pe_95_pwt = _percentile_error(95, x_truth, y_truth, x_pred, y_pred)
    
    # Output
    print("Mean: ", mean)
    print("Standard Deviation: ", stdev)
    print("MAE: ", mae)
    print("RMSE: ", rmse)
    print("75% Percentile Error Threshold: ", pe_75_et, "75% Percentile Percentage within Threshold: ", pe_75_pwt)
    print("90% Percentile Error Threshold: ", pe_90_et, "90% Percentile Percentage within Threshold: ", pe_90_pwt)
    print("95% Percentile Error Threshold: ", pe_95_et, "95% Percentile Percentage within Threshold: ", pe_95_pwt)
    
    
def _percentile_error(percentile, x_truth, y_truth, x_pred, y_pred):
    # Calculate Euclidean Distance
    errors = np.sqrt((np.array(x_truth) - np.array(x_pred))**2 + (np.array(y_truth) - np.array(y_pred))**2)
    
    # Sort
    sorted_errors = np.sort(errors)
    index = int(np.ceil((percentile / 100) * (len(errors) + 1)))
    
    # Calculate Metrics
    error_threshold = sorted_errors[index - 1]
    num_within_threshold = np.sum(errors <= error_threshold)
    percentage_within_threshold = (num_within_threshold / len(errors)) * 100
    
    return error_threshold, percentage_within_threshold

### Pipeline Execution

In [7]:
# TIME = 23.2s
geocoded_df = geocode(raw_df, "Location")

### Accuracy Assessment

In [8]:
# Results
preds = geocoded_df[["ICR", "x", "y"]].copy()

truth = truth_df[["ICR", "GroudTruth_X", "GroudTruth_Y"]].copy()

# Combining Predictions with Ground Truth
accuracy_df = pd.merge(preds, truth, on=["ICR"])

In [9]:
# Accuracy Assessment
accuracy_assessment(accuracy_df)

Mean:  0.1046584141414144
Standard Deviation:  1.138307128643687
MAE:  0.26427867575757547
RMSE:  1.1431082638016536
75% Percentile Error Threshold:  0.08489886644502057 75% Percentile Percentage within Threshold:  75.75757575757575
90% Percentile Error Threshold:  0.417088769933617 90% Percentile Percentage within Threshold:  90.9090909090909
95% Percentile Error Threshold:  3.3495781306810377 95% Percentile Percentage within Threshold:  95.95959595959596


In [10]:
# Reset Indexes & Ensure Invalid Records are Dropped from Truth DF
truth = truth[truth.ICR.isin(list(preds["ICR"])) == True]

truth = truth.reset_index()
preds = preds.reset_index()

# Convert to GDFs
gdf_pred = gpd.GeoDataFrame(
            preds, geometry=gpd.points_from_xy(preds["x"], preds["y"]), crs='EPSG:4326'
        )
gdf_truth = gpd.GeoDataFrame(
            truth, geometry=gpd.points_from_xy(truth["GroudTruth_X"], truth["GroudTruth_Y"]), crs='EPSG:4326'
        )

# Create Map
error_map = folium.Map(location=[45.85, -94], tiles="Cartodb Positron", zoom_start=6.5)

# Create list of point gemoetries from GDF
pred_points = [[pt.xy[1][0], pt.xy[0][0]] for pt in gdf_pred.geometry]
truth_points = [[pt.xy[1][0], pt.xy[0][0]] for pt in gdf_truth.geometry]

# Add Markers to Map
def add_features(gdf, point_list, color):
    i = 0
    for feature in point_list:   
        error_map.add_child(
            folium.Marker(
                location = feature,
                popup = "ICR: " + str(gdf.ICR[i]),
                icon=folium.Icon(color="%s" % color, icon="exclamation-sign")
            )
        )
        i += 1
        
add_features(gdf_pred, pred_points, "blue")    
add_features(gdf_truth, truth_points, "green") 

# Visualize
error_map